In [75]:
!pip install git+https://github.com/GTSF-Quantitative-Sector/sec_parser.git
!pip install polygon-api-client
!pip install scipy

  Cloning https://github.com/GTSF-Quantitative-Sector/sec_parser.git to /private/var/folders/n1/my7m6zj901718by459kvh6z00000gn/T/pip-req-build-5bl7ng0b
  Running command git clone --filter=blob:none --quiet https://github.com/GTSF-Quantitative-Sector/sec_parser.git /private/var/folders/n1/my7m6zj901718by459kvh6z00000gn/T/pip-req-build-5bl7ng0b
  Resolved https://github.com/GTSF-Quantitative-Sector/sec_parser.git to commit 3bde3fd9b289cbcf2a38fac9790c0fbf2f47b962
  Preparing metadata (setup.py) ... done


In [133]:
from sec import constants, stock, lookups, processor
import pandas as pd
from datetime import datetime, timedelta, date
from typing import Iterator
import asyncio
from tqdm import tqdm
import pickle
import os
import matplotlib.pyplot as plt

In [77]:
constants.set_polygon_key("_4BtZn3PRCLu6fsdu7dgddb4ucmB1sfp")

In [78]:
API_KEY = "uwQtl3txGt5BLbecq7ZbIu0ZbuitCGjc"

In [79]:
risk_free_interest_rate = 4.11

In [80]:
# Get the current date
def get_date(day: int = 0):
# Format the date as YYYY-MM-DD
  today = datetime.now()
  day = today - timedelta(days=day)
  date = day.strftime('%Y-%m-%d')
  return date

In [81]:
# Hard coded every industry's Beta
industry_beta = {
  "Advertising": 1.37,
  "Aerospace/Defense": 1.08,
  "Air Transport": 1.27,
  "Apparel": 1.19,
  "Auto & Truck": 1.52,
  "Auto Parts": 1.34,
  "Bank (Money Center)": 1.06,
  "Banks (Regional)": 0.46,
  "Beverage (Alcoholic)": 1.13,
  "Beverage (Soft)": 0.76,
  "Broadcasting": 1.06,
  "Brokerage & Investment Banking": 1.12,
  "Building Materials": 1.32,
  "Business & Consumer Services": 1.02,
  "Cable TV": 1.28,
  "Chemical (Basic)": 1.1,
  "Chemical (Diversified)": 1.13,
  "Chemical (Specialty)": 1.09,
  "Coal & Related Energy": 1.27,
  "Computer Services": 1.0,
  "Computers/Peripherals": 1.13,
  "Construction Supplies": 0.94,
  "Diversified": 1.19,
  "Drugs (Biotechnology)": 1.12,
  "Drugs (Pharmaceutical)": 1.03,
  "Education": 1.23,
  "Electrical Equipment": 1.24,
  "Electronics (Consumer & Office)": 1.3,
  "Electronics (General)": 0.93,
  "Engineering/Construction": 1.06,
  "Entertainment": 0.99,
  "Environmental & Waste Services": 0.91,
  "Farming/Agriculture": 0.99,
  "Financial Svcs. (Non-bank & Insuran": 1.14,
  "Food Processing": 0.71,
  "Food Wholesalers": 0.97,
  "Furn/Home Furnishings": 1.11,
  "Green & Renewable Energy": 1.11,
  "Healthcare Products": 1.06,
  "Healthcare Support Services": 1.03,
  "Heathcare Information and Technol": 1.27,
  "Homebuilding": 1.37,
  "Hospitals/Healthcare Facilities": 0.88,
  "Hotel/Gaming": 1.34,
  "Household Products": 0.78,
  "Information Services": 0.93,
  "Insurance (General)": 1.03,
  "Insurance (Life)": 0.77,
  "Insurance (Prop/Cas.)": 0.74,
  "Investments & Asset Management": 0.46,
  "Machinery": 1.03,
  "Metals & Mining": 0.96,
  "Office Equipment & Services": 1.14,
  "Oil/Gas (Integrated)": 0.67,
  "Oil/Gas (Production and Exploratio": 0.93,
  "Oil/Gas Distribution": 0.79,
  "Oilfield Svcs/Equip.": 0.98,
  "Packaging & Container": 0.87,
  "Paper/Forest Products": 1.94,
  "Power": 0.65,
  "Precious Metals": 0.87,
  "Publishing & Newspapers": 0.92,
  "R.E.I.T.": 1.02,
  "Real Estate (Development)": 0.67,
  "Real Estate (General/Diversified)": 0.56,
  "Real Estate (Operations & Services)": 1.08,
  "Recreation": 1.17,
  "Reinsurance": 0.66,
  "Restaurant/Dining": 1.19,
  "Retail (Automotive)": 1.49,
  "Retail (Building Supply)": 1.94,
  "Retail (Distributors)": 1.11,
  "Retail (General)": 1.25,
  "Retail (Grocery and Food)": 0.49,
  "Retail (REITs)": 1.12,
  "Retail (Special Lines)": 1.18,
  "Rubber& Tires": 0.67,
  "Semiconductor": 1.5,
  "Semiconductor Equip": 1.53,
  "Shipbuilding & Marine": 0.81,
  "Shoe": 1.29,
  "Software (Entertainment)": 1.11,
  "Software (Internet)": 1.62,
  "Software (System & Application)": 1.29,
  "Steel": 1.13,
  "Telecom (Wireless)": 1.09,
  "Telecom. Equipment": 1.08,
  "Telecom. Services": 0.78,
  "Tobacco": 1.22,
  "Transportation": 1.0,
  "Transportation (Railroads)": 1.02,
  "Trucking": 1.15,
  "Utility (General)": 0.58,
  "Utility (Water)": 0.71,
  "Total Market": 1.0,
  "Total Market (without financials)": 1.1
}

In [82]:
# Confining industry_beta dictionary to all only industries with Betas greater than 1.2
keys = list(industry_beta.keys())
for key in keys:
  if (abs(industry_beta[key]) < 1.19):
    del industry_beta[key]
for industry in industry_beta:
  print("Industry: " + str(industry) + " Beta: " + str(industry_beta[industry]))


Industry: Advertising Beta: 1.37
Industry: Air Transport Beta: 1.27
Industry: Apparel Beta: 1.19
Industry: Auto & Truck Beta: 1.52
Industry: Auto Parts Beta: 1.34
Industry: Building Materials Beta: 1.32
Industry: Cable TV Beta: 1.28
Industry: Coal & Related Energy Beta: 1.27
Industry: Diversified Beta: 1.19
Industry: Education Beta: 1.23
Industry: Electrical Equipment Beta: 1.24
Industry: Electronics (Consumer & Office) Beta: 1.3
Industry: Heathcare Information and Technol Beta: 1.27
Industry: Homebuilding Beta: 1.37
Industry: Hotel/Gaming Beta: 1.34
Industry: Paper/Forest Products Beta: 1.94
Industry: Restaurant/Dining Beta: 1.19
Industry: Retail (Automotive) Beta: 1.49
Industry: Retail (Building Supply) Beta: 1.94
Industry: Retail (General) Beta: 1.25
Industry: Semiconductor Beta: 1.5
Industry: Semiconductor Equip Beta: 1.53
Industry: Shoe Beta: 1.29
Industry: Software (Internet) Beta: 1.62
Industry: Software (System & Application) Beta: 1.29
Industry: Tobacco Beta: 1.22


In [188]:
print (get_date(87))

2024-01-01


In [189]:
import polygon

## Finds stocks with a realized volatility that is greater than their Industry's Beta from the past day
async def find_stocks()-> (dict):
  # Returns the sotcks to buy and the date of when the purchase was made
  # Gets all S&P500 stocks
  curr_sp = lookups.get_sp500_tickers()
  client = polygon.OptionsClient(API_KEY)
  print ("Curr SP: " + str(curr_sp))
  
  # Gets the YYYY-MM-DD value of Today and Yesterday
  today = '2024-01-02'
  yesterday = '2024-01-01'
  
  # parts = today.split('-')
  # transformed_date = parts[0][2:] + parts[1] + parts[2]
  print ("Today: " + today)
  print ("Yesterday: " + yesterday)
 
  # Gets the percent change of the price of SPY from yesterday to today
  spy = stock.Stock(str("SPY"))
  todays_price = await spy.get_price(today)
  yesterdays_price = await spy.get_price(yesterday)
  spy_price_change = (abs((todays_price - yesterdays_price)/yesterdays_price)) * 100
  print ("Spy Price Change: " + str(spy_price_change))
  # Stores the stocks we are going to purchase options form and their prices.
  buy_options = {}

  # Keeps track of number of stocks added to buy_stocks. Limiting to 5**
  i = 0

  # Runs through every stock in the S&P 500
  for ticker in curr_sp:

    try:
      # Allows for us to get the price and industry of ticker
      tick = stock.Stock(str(ticker))
      industry = lookups.get_industry(ticker)
    
      # We want the industry beta to be greater than 1 to ensure that it is volatile in order to make gamma scalping possible
      if (industry not in industry_beta):
          continue

      try:
      # Tries to get the price of the stock from today and yesterday
        todays_price = await tick.get_price(today)
        yesterdays_price = await tick.get_price(yesterday)
      except:
      # Skips stock if price is not available
        print ("Error: Couldnt get the price")
        continue

      # Gets the percent change of the price from yesterday to today
      price_change = (abs((todays_price - yesterdays_price)/yesterdays_price)) * 100

      # 1. Divides the percent change in price of the stock divided by the industry beta. We should do this because if the beta is 1.5 than
      # for every percent change in SPY the respective stock should have changed by 1.5. This will allow us to see if the stock actually increased
      # more than 1.5% per SPY's 1%.
      # 2. Multiplies Spy's percent change in price by 15 to find the stocks that have a huge difference in percent increase from their industry Beta.
      # 3. Only accounting for 5 stocks currently, but this could be changed. I just wanted simplicity
      # 4. Goal for future: Want to use Implied Volatility of a stock instead of Beta so we can compare Realized vs Implied volatility from a current day.

      if (price_change/industry_beta[industry] > spy_price_change * 3 and i < 5):
        todays_price = int(todays_price)
        strike_price = round(todays_price / 5) * 5
        friday = '2024-02-02'
        parts = friday.split('-')
        transformed_date = parts[0][2:] + parts[1] + parts[2]
        symbol_put = polygon.build_option_symbol(ticker, transformed_date, 'put', strike_price)
 
        
        option_put = client.get_daily_open_close(symbol_put, friday)
        

        symbol_call = polygon.build_option_symbol(ticker, transformed_date, 'call', strike_price)
  

        option_call = client.get_daily_open_close(symbol_call, friday)
       
        

        if (option_put['status'] != 'OK' or option_call['status'] != 'OK'):
          print (ticker + " Put: " + str(option_put['status']) + " Call: " + str(option_call['status']))
          continue
        else:
          buy_options[ticker] = ((todays_price, strike_price, option_call['close'], option_put['close']))
          print (str(option_call))
          print (str(option_put))
          i += 1
        # Once 5 stocks are found return the stock options we want to buy
        if (i == 5):
          return (buy_options)
    except Exception as e:
      print(f"Error processing {ticker}: {str(e)}")
      #Stock not found in SEC Parser
      continue;
  return buy_options

In [190]:
buy_options = await find_stocks()


Curr SP: ['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABT', 'ACGL', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEP', 'AES', 'AFL', 'AIG', 'AIZ', 'AJG', 'AKAM', 'ALB', 'ALGN', 'ALK', 'ALL', 'ALLE', 'AMAT', 'AMCR', 'AMD', 'AME', 'AMGN', 'AMP', 'AMT', 'AMZN', 'ANET', 'ANSS', 'AON', 'AOS', 'APA', 'APD', 'APH', 'APTV', 'ARE', 'ATO', 'ATVI', 'AVB', 'AVGO', 'AVY', 'AWK', 'AXON', 'AXP', 'AZO', 'BA', 'BAC', 'BALL', 'BAX', 'BBWI', 'BBY', 'BDX', 'BEN', 'BF.B', 'BG', 'BIIB', 'BIO', 'BK', 'BKNG', 'BKR', 'BLK', 'BMY', 'BR', 'BRK.B', 'BRO', 'BSX', 'BWA', 'BXP', 'C', 'CAG', 'CAH', 'CARR', 'CAT', 'CB', 'CBOE', 'CBRE', 'CCI', 'CCL', 'CDAY', 'CDNS', 'CDW', 'CE', 'CEG', 'CF', 'CFG', 'CHD', 'CHRW', 'CHTR', 'CI', 'CINF', 'CL', 'CLX', 'CMA', 'CMCSA', 'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COF', 'COO', 'COP', 'COST', 'CPB', 'CPRT', 'CPT', 'CRL', 'CRM', 'CSCO', 'CSGP', 'CSX', 'CTAS', 'CTLT', 'CTRA', 'CTSH', 'CTVA', 'CVS', 'CVX', 'CZR', 'D', 'DAL', 'DD', 'DE', 'DFS', 'DG', 'DGX', 'DHI', 'DHR', 'DIS'

In [171]:
print (buy_options)

{'AAL': (13, 15.0, 0.01, 0.35), 'AMAT': (154, 155.0, 13.12, 0.01), 'AMD': (138, 140.0, 37.78, 0.01), 'AVGO': (1085, 1085.0, 138.78, 0.01), 'CCL': (17, 15.0, 1.5, 0.01)}


In [86]:
# Next Steps in Creating Gamma Scalping Strategy:
# **Note: Using Polygon API for the options is going to be how we complete most of this work, so make sure you look inot their API docs and use all the correct end points.**
#     1. Determine how much stock we need to buy to become delta nuetral when buying options
#           - For every call or put option you plan to buy, you consider it as buying 10 of those options.
#           - To hedge a long call position, you buy puts. To hedge a long put position, you buy stocks.
#           - Your position's delta adjustment is calculated as: 10×Delta×100, where 10 is the multiplier for
#           the number of options, Delta is the option's delta, and 100 represents the standard number of
#           shares per option contract.
#           - Data Required:
#             - Delta --> For Calls: Delta ranges from 0 to 1. For Puts: Delta ranges from -1 to 0.
#             - Stock Price --> The current price of the underlying stock. This is crucial for calculating
#               the financial implications of buying or selling the stock to hedge your options. This will
#               be provided by the find_stocks fuction which will return a dictionary of stocks and their prices.

#     2. Determine when an option is above/below the strike price for calls/puts respectively
#.        - At the end of each day, look at closing price and see if it is above or below strike price respectively
#         - Data Required:
#           - Strike Price
#     3. If call/put is above/below the strike price we need to also buy stock or puts
#         - In order to remain delta nuetral we need to recalculate our exposure to delta and see how many stocks/puts we need to buy
#.          when we are in the money(above/below strike price).
#         - Data Required:
#.          - new Delta --> For Calls: Delta ranges from 0 to 1. For Puts: Delta ranges from -1 to 0.
#           - New Stock Price for puts/ New Put price for calls --> For hedging
#     4. If call/put goes below/above strike price respectively we need to now sell our stocks or puts
#         - In order to remain delta nuetral we need to recalculate our exposure to delta and see how many stocks/puts 
#            we need to sell when we become out of the money(below/above strike price).
#         - Data Required:
#.          - new Delta --> For Calls: Delta ranges from 0 to 1. For Puts: Delta ranges from -1 to 0.
#           - New Stock Price for puts/ New Put price for calls --> For hedging
#     5. Once there is a day before expiration we do not want ot dno any more scalping sicne there is a lot of weird things that go on with options on expirary


# Back Test:
# When creating the back test, we will use these algorithms. Obviously we do not have the ability to asynchronously make these trades. So the back test will allow us to
# pretend that we are making these trades "asynchronously". So for right now our objective is to make sure our logic works so we can fine tune it once we start testing it out.
#
#

In [182]:
import numpy as np
from scipy.stats import norm
from scipy.special import *
N = norm.cdf

def bs_call(S, K, T, r, vol):
    d1 = (np.log(S/K) + (r + 0.5*vol**2)*T) / (vol*np.sqrt(T))
    d2 = d1 - vol * np.sqrt(T)
    return S * ndtr(d1) - np.exp(-r * T) * K * ndtr(d2)

def bs_put(S, K, T, r, vol):
    d1 = (np.log(S/K) + (r + 0.5*vol**2)*T) / (vol*np.sqrt(T))
    d2 = d1 - vol * np.sqrt(T)
    return K * np.exp(-r * T) * ndtr(-d2) - S * ndtr(-d1)

def bs_vega(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    return S * norm._pdf(d1) * np.sqrt(T)

def get_delta(S, K, T, r, sigma, call_put):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    if (call_put == 'call'):
        return norm.cdf(d1)
    else :
        return norm.cdf(d1) - 1

def find_vol(target_value, S, K, T, r, call_put):
    MAX_ITERATIONS = 200
    PRECISION = 1.0e-5
    sigma = 0.5
    if (call_put == 'call'):
        for i in range(0, MAX_ITERATIONS):
            price = bs_call(S, K, T, r, sigma)
            vega = bs_vega(S, K, T, r, sigma)
            if (vega < 1e-5):
                return sigma
            diff = target_value - price  # our root
            if (abs(diff) < PRECISION):
                return sigma
            sigma = sigma + diff/vega # f(x) / f'(x)
    else :
        for i in range(0, MAX_ITERATIONS):
            
            price = bs_put(S, K, T, r, sigma)
            vega = bs_vega(S, K, T, r, sigma)
            if (vega < 1e-5):
                return sigma
            diff = target_value - price  # our root
            if (abs(diff) < PRECISION):
                return sigma
            sigma = sigma + diff/vega # f(x) / f'(x)
            
    return sigma # value wasn't found, return best guess so far

In [191]:

for ticker, data in buy_options.items():
    S = data[0]
    K = data[1]
    T = 1/12
    r = .1
    
    
    implied_vol = find_vol(data[2], S, K, T, r, 'call')
    delta = get_delta(S,K,T,r,implied_vol,'call')


    print (str(ticker)+ ' Call Implied vol: %.2f%%' % (implied_vol * 100))
    print (str(ticker)+ ' Call Delta: ' + str(delta))

   
    S = data[0]
    K = data[1]
    T = 1/12
    r = .1
    
    
    implied_vol = find_vol(data[2], S, K, T, r, 'put')
    delta = get_delta(S,K,T,r,implied_vol,'put')
    
    print (str(ticker)+ ' Put Implied vol: %.2f%%' % (implied_vol * 100))
    print (str(ticker)+ ' Put Delta: ' + str(delta))

   

AAL Call Implied vol: 24.22%
AAL Call Delta: 0.02921117287117341
AAL Put Implied vol: -9.31%
AAL Put Delta: -2.840636852408096e-07
AMAT Call Implied vol: 73.37%
AMAT Call Delta: 0.5456538577755208
AMAT Put Implied vol: 75.00%
AMAT Put Delta: -0.4534920405085956
AMD Call Implied vol: 244.61%
AMD Call Delta: 0.6347593203214646
AMD Put Implied vol: 239.02%
AMD Put Delta: -0.3683530564282844
AVGO Call Implied vol: 108.31%
AVGO Call Delta: 0.5725956841488775
AVGO Put Implied vol: 115.65%
AVGO Put Delta: -0.42391751074015804
CCL Call Implied vol: -1.58%
CCL Call Delta: 8.718490346359995e-188
CCL Put Implied vol: 132.58%
CCL Put Delta: -0.294538894234745


In [127]:
import datetime
import pandas as pd
import numpy as np
import polygon

def find_annualized_volatility():
    annualized_volatility_dict = {}
    client = polygon.StocksClient(API_KEY)

    for ticker in buy_options:
        print(ticker)
        price = []
        start_date = datetime.date(2023, 1, 1)
        end_date = datetime.date(2023, 12, 31)
        current_date = start_date
        while current_date <= end_date: 
            if (current_date.weekday() > 0 and current_date.weekday() < 5):    
                try:
                    # Tries to get the price of the stock for the current date
                    prices = client.get_daily_open_close(ticker, current_date)
                    if prices:
                        # Append the closing price to the price list
                        price.append(prices['close'])
                    else:
                        print(f"Error: No price data available for {ticker} on {current_date}")
                except Exception as e:
                    # Handle any errors that occur during data retrieval
                    print(f"Error fetching data for {ticker} on {current_date}: {e}")
                    current_date += datetime.timedelta(days=1)
                    continue

                current_date += datetime.timedelta(days=1)
            else:
                current_date += datetime.timedelta(days=1)

        # Calculate daily returns and annualized volatility if price data is available
        if price:
 
            try:
                # Convert price list to pandas Series
                price_series = pd.Series(price)
                # Calculate daily returns
                returns = price_series.pct_change()
   
                # Calculate standard deviation of daily returns
                std_dev = np.std(returns)
      
                # Annualize volatility
                trading_days_per_year = 252  # Typical number of trading days in a year
                annualized_volatility = std_dev * np.sqrt(trading_days_per_year)
                
                # Add annualized volatility to the dictionary
                annualized_volatility_dict[ticker] = annualized_volatility
            except Exception as e:
                # Handle any errors that occur during volatility calculation
                print(f"Error calculating volatility for {ticker}: {e}")
        else:
                print(f"No price data available for {ticker}")

    return annualized_volatility_dict

# Usage example



In [110]:
annualized_volatility = find_annualized_volatility()

GNRC
Error fetching data for GNRC on 2023-04-07: 'close'
Error fetching data for GNRC on 2023-07-04: 'close'
Error fetching data for GNRC on 2023-11-23: 'close'
NKE
Error fetching data for NKE on 2023-04-07: 'close'
Error fetching data for NKE on 2023-07-04: 'close'
Error fetching data for NKE on 2023-11-23: 'close'
NVDA
Error fetching data for NVDA on 2023-04-07: 'close'
Error fetching data for NVDA on 2023-07-04: 'close'
Error fetching data for NVDA on 2023-11-23: 'close'
SEDG
Error fetching data for SEDG on 2023-04-07: 'close'
Error fetching data for SEDG on 2023-07-04: 'close'
Error fetching data for SEDG on 2023-11-23: 'close'
VFC
Error fetching data for VFC on 2023-04-07: 'close'
Error fetching data for VFC on 2023-07-04: 'close'
Error fetching data for VFC on 2023-11-23: 'close'
